In [ ]:
# Setup and imports
import sys
import os
sys.path.append('..')
sys.path.append('../src')

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import asyncio
import logging
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")


In [ ]:
# Import our custom modules
print("🔄 Attempting to import custom modules...")

# Add project paths to sys.path
import sys
import os
project_root = os.path.abspath('.')
src_path = os.path.join(project_root, 'src')
data_path = os.path.join(project_root, 'data')

for path in [project_root, src_path, data_path]:
    if path not in sys.path:
        sys.path.insert(0, path)

print(f"📁 Added paths: {[project_root, src_path, data_path]}")

try:
    # First try direct imports
    from models.lstm_model import LSTMPricePredictor, create_lstm_model
    from models.rl_agent import TradingAgent, DQNAgent, TradingEnvironment
    from trading.live_trading import LiveTradingSystem, LLMMarketAnalyst, MarketDataProvider
    from features.build_features import build_dataset
    from data.fetch_data import get_stock_data
    print("✅ Direct imports successful!")
    import_method = "direct"
except ImportError as e1:
    print(f"⚠️ Direct imports failed: {e1}")
    # If that fails, try with src prefix
    try:
        from src.models.lstm_model import LSTMPricePredictor, create_lstm_model
        from src.models.rl_agent import TradingAgent, DQNAgent, TradingEnvironment
        from src.trading.live_trading import LiveTradingSystem, LLMMarketAnalyst, MarketDataProvider
        from src.features.build_features import build_dataset
        from src.data.fetch_data import get_stock_data
        print("✅ Src imports successful!")
        import_method = "src_prefix"
    except ImportError as e2:
        print(f"⚠️ Src imports also failed: {e2}")
        print("🔧 Creating fallback implementations...")
        
        import torch
        import torch.nn as nn
        import numpy as np
        import pandas as pd
        from typing import Tuple
        
        # Create fallback implementations
        class LSTMPricePredictor(nn.Module):
            def __init__(self, input_size=10, hidden_size=128, num_layers=2, dropout=0.2, sequence_length=60):
                super().__init__()
                self.sequence_length = sequence_length
                self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
                self.fc = nn.Linear(hidden_size, 1)
                
            def forward(self, x):
                lstm_out, _ = self.lstm(x)
                features = lstm_out[:, -1, :]
                prediction = self.fc(features)
                return prediction, features
                
            def extract_features(self, x):
                _, features = self.forward(x)
                return features
        
        def create_lstm_model(config):
            return LSTMPricePredictor(**config)
        
        class TradingEnvironment:
            def __init__(self, initial_balance=10000):
                self.initial_balance = initial_balance
                self.reset()
                
            def reset(self):
                self.balance = self.initial_balance
                self.position = 0.0
                self.portfolio_value = self.initial_balance
                return {'balance': self.balance, 'position': self.position, 'portfolio_value': self.portfolio_value, 'step_count': 0}
        
        class DQNAgent(nn.Module):
            def __init__(self, state_size=131, action_size=3):
                super().__init__()
                self.network = nn.Sequential(
                    nn.Linear(state_size, 256),
                    nn.ReLU(),
                    nn.Dropout(0.2),
                    nn.Linear(256, 128),
                    nn.ReLU(),
                    nn.Dropout(0.2),
                    nn.Linear(128, action_size)
                )
                
            def forward(self, x):
                return self.network(x)
                
            def act(self, state, training=False):
                if isinstance(state, np.ndarray):
                    state_tensor = torch.FloatTensor(state).unsqueeze(0)
                else:
                    state_tensor = state.unsqueeze(0) if len(state.shape) == 1 else state
                q_values = self.forward(state_tensor)
                return np.argmax(q_values.detach().numpy())
        
        class TradingAgent:
            def __init__(self, lstm_model, dqn_agent, environment):
                self.lstm_model = lstm_model
                self.dqn_agent = dqn_agent
                self.environment = environment
                
            def prepare_state(self, lstm_features, env_state):
                portfolio_features = np.array([
                    env_state['balance'] / self.environment.initial_balance,
                    env_state['position'],
                    env_state['portfolio_value'] / self.environment.initial_balance
                ])
                return np.concatenate([lstm_features.flatten(), portfolio_features])
                
            def predict_action(self, lstm_features, env_state):
                state = self.prepare_state(lstm_features, env_state)
                action = self.dqn_agent.act(state, training=False)
                action_names = ['HOLD', 'BUY', 'SELL']
                return action, action_names[action]
        
        # Dummy data functions with realistic market behavior
        def get_stock_data(symbol, start, end):
            dates = pd.date_range(start, end, freq='D')
            n_days = len(dates)
            
            # Generate realistic stock price data
            base_price = 150 if symbol == 'AAPL' else 100
            returns = np.random.normal(0.001, 0.02, n_days)  # Daily returns
            prices = [base_price]
            
            for i in range(1, n_days):
                price = prices[-1] * (1 + returns[i])
                prices.append(max(price, 1))  # Ensure positive prices
            
            # Create OHLC data
            data = {
                'Open': [p * np.random.uniform(0.99, 1.01) for p in prices],
                'High': [p * np.random.uniform(1.00, 1.03) for p in prices],
                'Low': [p * np.random.uniform(0.97, 1.00) for p in prices],
                'Close': prices,
                'Volume': np.random.randint(1000000, 50000000, size=n_days)
            }
            
            df = pd.DataFrame(data, index=dates)
            df.index.name = 'Date'
            return df
        
        def build_dataset(data):
            # Comprehensive feature engineering
            features = data.copy()
            
            # Moving averages
            features['MA_5'] = features['Close'].rolling(5).mean()
            features['MA_20'] = features['Close'].rolling(20).mean()
            
            # RSI calculation
            delta = features['Close'].diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
            rs = gain / loss
            features['RSI'] = 100 - (100 / (1 + rs))
            
            # Price change and volume features
            features['Price_Change'] = features['Close'].pct_change()
            features['Volume_Ratio'] = features['Volume'] / features['Volume'].rolling(20).mean()
            
            # Bollinger Bands
            bb_window = 20
            bb_std = features['Close'].rolling(window=bb_window).std()
            bb_mean = features['Close'].rolling(window=bb_window).mean()
            features['BB_Upper'] = bb_mean + (bb_std * 2)
            features['BB_Lower'] = bb_mean - (bb_std * 2)
            
            # Select final features
            feature_columns = ['Close', 'Volume', 'MA_5', 'MA_20', 'RSI', 'Price_Change', 
                              'Volume_Ratio', 'BB_Upper', 'BB_Lower', 'Open']
            return features[feature_columns].dropna()
        
        # Placeholder classes for live trading (not functional without real implementation)
        class LiveTradingSystem:
            def __init__(self, *args, **kwargs):
                print("⚠️ LiveTradingSystem placeholder - real implementation needed")
        
        class LLMMarketAnalyst:
            def __init__(self, *args, **kwargs):
                print("⚠️ LLMMarketAnalyst placeholder - real implementation needed")
        
        class MarketDataProvider:
            def __init__(self, *args, **kwargs):
                print("⚠️ MarketDataProvider placeholder - real implementation needed")
        
        print("✅ Fallback implementations created!")
        import_method = "fallback"

print(f"✅ Module import completed using: {import_method}")
print("🚀 Ready to proceed with the demo!")


In [ ]:
# Configuration
CONFIG = {
    'symbols': ['AAPL', 'GOOGL', 'MSFT', 'TSLA'],
    'initial_balance': 10000.0,
    'openai_api_key': 'your-openai-api-key-here',
    'model_config': {
        'input_size': 10,
        'hidden_size': 128,
        'num_layers': 2,
        'dropout': 0.2,
        'sequence_length': 60
    }
}

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ Configuration loaded!")
print(f"Trading symbols: {CONFIG['symbols']}")
print(f"Initial balance: ${CONFIG['initial_balance']:,.2f}")


In [ ]:
# Load sample data for demonstration
symbol = 'AAPL'  # Focus on Apple for demo
print(f"Loading data for {symbol}...")

# Get historical data
end_date = datetime.now().strftime('%Y-%m-%d')
recent_start = (datetime.now() - timedelta(days=5)).strftime('%Y-%m-%d')
historical_start = (datetime.now() - timedelta(days=90)).strftime('%Y-%m-%d')

historical_data = get_stock_data(symbol, historical_start, end_date)
recent_data = get_stock_data(symbol, recent_start, end_date)

print(f"Loaded {len(historical_data)} days of historical data")
print(f"Loaded {len(recent_data)} days of recent data")

# Display data info
print("\\n📈 Recent Price Data:")
print(recent_data.tail())

# Plot recent price action
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Price chart
ax1.plot(recent_data.index, recent_data['Close'], label='Close Price', linewidth=2)
ax1.fill_between(recent_data.index, recent_data['Low'], recent_data['High'], alpha=0.3, label='Daily Range')
ax1.set_title(f'{symbol} - Recent Price Action', fontsize=14, fontweight='bold')
ax1.set_ylabel('Price ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Volume chart
ax2.bar(recent_data.index, recent_data['Volume'], alpha=0.7, color='orange')
ax2.set_title('Trading Volume', fontsize=12)
ax2.set_ylabel('Volume')
ax2.set_xlabel('Date')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
